In [1]:
from collections import namedtuple
import itertools
import math

In [2]:
PointSet = namedtuple("PointSet", " points op val")

In [3]:
BOARD_SIZE=3

operators = 'DIV', 'PROD', 'SUM', 'SUB'

sets = [
    PointSet( ((0,0), (1,0)), 'DIV', 2 ),
    PointSet( ((2,0), (2,1)), 'SUB', 2 ),
    PointSet( ((0,1), (0,2)), 'PROD', 6 ),
    PointSet( ((1,1), (1,2), (2,2)), 'SUM', 6 )

]

sets


[PointSet(points=((0, 0), (1, 0)), op='DIV', val=2),
 PointSet(points=((2, 0), (2, 1)), op='SUB', val=2),
 PointSet(points=((0, 1), (0, 2)), op='PROD', val=6),
 PointSet(points=((1, 1), (1, 2), (2, 2)), op='SUM', val=6)]

In [4]:
def get_factors(num, max_factor):
    return [i for i in range(1, max_factor+1) if num%i == 0]
    

In [5]:
def enum_set_solutions(ptset):
    solutions = {}
    if ptset.op == 'DIV':
        solutions = [(i, i*ptset.val) for i in range(1, int(BOARD_SIZE/ptset.val)+1)]
    if ptset.op == 'SUB':
        solutions = [(i, i+ptset.val) for i in range(1, BOARD_SIZE-ptset.val+1)]
    if ptset.op == 'PROD':
        factors = get_factors(ptset.val, BOARD_SIZE)
        combos = itertools.product(factors, repeat=len(ptset.points))
        possibles = [tuple(sorted(combo)) for combo in combos if math.prod(combo)==ptset.val]
        solutions = list(set(possibles))
    if ptset.op == 'SUM':
        possible_nums = range(1,BOARD_SIZE+1)
        combos = itertools.product(possible_nums, repeat=len(ptset.points))
        possibles = [tuple(sorted(combo)) for combo in combos if sum(combo)==ptset.val]
        solutions = list(set(possibles))

    return solutions

In [6]:
[enum_set_solutions(s) for s in sets]

[[(1, 2)], [(1, 3)], [(2, 3)], [(1, 2, 3), (2, 2, 2)]]